### Visualizing the distribution of the observations

### Load the required libraries

In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import urllib
import math
%pylab inline

import findspark
findspark.init()

from pyspark import SparkContext
#sc.stop()
sc = SparkContext(master="local[3]",pyFiles=['lib/numpy_pack.py','lib/spark_PCA.py','lib/computeStats.py'])

from pyspark import SparkContext
from pyspark.sql import *
sqlContext = SQLContext(sc)

import sys
sys.path.append('./lib')

import numpy as np
from numpy_pack import packArray,unpackArray
from spark_PCA import computeCov
from computeStats import computeOverAllDist, STAT_Descriptions

### Read the data frame from pickle file

data_dir='../../Data/Weather'
file_index='SSSBSSSS'

from pickle import load

#read statistics
filename=data_dir+'/STAT_%s.pickle'%file_index
STAT,STAT_Descriptions = load(open(filename,'rb'))
#print 'keys from STAT=',STAT.keys()

#read data
filename=data_dir+'/US_Weather_%s.parquet'%file_index

df=sqlContext.read.parquet(filename)
print df.count()
df.show(5)

Populating the interactive namespace from numpy and matplotlib
12741
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|elevation|latitude|longitude|measurement|    station|undefs|              vector|  year|   label|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     2|[B8 59 00 58 38 5...|2002.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     2|[90 59 10 5A 20 5...|2003.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     1|[E0 55 00 58 E0 5...|2004.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     2|[50 57 90 55 A0 5...|2005.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     7|[A0 56 F0 56 A0 5...|2006.0|SSSBSSSS|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
only showing top 5 rows



### Select data for a particular station and measurement type

In [2]:
sqlContext.registerDataFrameAsTable(df,'weather')
Query1="SELECT * FROM weather\n\tWHERE measurement='%s'"%('TMAX')
print Query1
df_test = sqlContext.sql(Query1)
print df_test.count(),'rows'
df_test.show()
rows_test=df_test.rdd.map(lambda row:(row.elevation,row.station,unpackArray(row['vector'],np.float16))).collect()
#rows_test1=rows_test.map(lambda (v,y):(v,np.hstack(y))).collect()
 #xs: [x[0] for x in xs]

                         #T=np.vstack(rows_test
#T=T/10.  # scaling to make the temperature be in centingrates
#shape(T)


import pandas as pd

df_test=pd.DataFrame(rows_test)

df_test1=df_test[2].apply(pd.Series)/10

# rename 0 to elvation 

del df_test[2]

df_test3=pd.concat([df_test,df_test1],axis=1)

df_test3

SELECT * FROM weather
	WHERE measurement='TMAX'
2179 rows
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|elevation|latitude|longitude|measurement|    station|undefs|              vector|  year|   label|
+---------+--------+---------+-----------+-----------+------+--------------------+------+--------+
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     2|[B8 59 00 58 38 5...|2002.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     2|[90 59 10 5A 20 5...|2003.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     1|[E0 55 00 58 E0 5...|2004.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     2|[50 57 90 55 A0 5...|2005.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     7|[A0 56 F0 56 A0 5...|2006.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CPNN|     2|[80 58 B8 59 C8 5...|2007.0|SSSBSSSS|
|    402.9| 36.4708|-121.1472|       TMAX|USR0000CP

,0,1,0,1,2,3,4,5,6,7,...,355,356,357,358,359,360,361,362,363,364
0,402.9,USR0000CPNN,18.296875,12.796875,16.703125,16.093750,17.796875,17.796875,18.296875,17.796875,...,10.000000,13.296875,9.398438,11.703125,11.101562,14.398438,12.796875,12.203125,11.703125,13.296875
1,402.9,USR0000CPNN,17.796875,19.406250,22.796875,20.593750,20.593750,22.796875,25.593750,21.703125,...,13.898438,11.703125,12.203125,7.800781,10.000000,12.203125,11.703125,7.800781,15.000000,15.601562
2,402.9,USR0000CPNN,9.398438,12.796875,9.398438,11.703125,15.000000,12.203125,16.093750,21.703125,...,15.601562,17.796875,18.296875,17.203125,10.601562,12.203125,8.296875,12.796875,11.101562,12.203125
3,402.9,USR0000CPNN,11.703125,8.898438,12.203125,10.601562,10.601562,13.296875,13.296875,11.101562,...,19.406250,22.203125,25.593750,15.601562,16.703125,18.296875,14.398438,12.203125,15.000000,13.296875
4,402.9,USR0000CPNN,10.601562,11.101562,12.203125,21.703125,24.406250,25.000000,16.703125,15.601562,...,13.296875,17.203125,20.000000,22.796875,16.093750,10.000000,11.101562,15.000000,20.000000,10.000000
5,402.9,USR0000CPNN,14.398438,18.296875,21.703125,11.703125,10.000000,11.703125,18.296875,22.796875,...,15.601562,17.796875,14.398438,16.093750,10.601562,9.398438,11.101562,11.703125,11.703125,17.203125
6,402.9,USR0000CPNN,23.296875,20.000000,13.296875,11.703125,11.703125,9.398438,10.000000,8.898438,...,11.703125,12.796875,9.398438,9.398438,10.601562,15.000000,16.703125,17.203125,21.093750,17.796875
7,402.9,USR0000CPNN,20.000000,17.796875,10.601562,15.601562,8.296875,16.093750,21.703125,18.296875,...,10.601562,16.093750,17.796875,21.093750,12.203125,16.093750,13.296875,16.093750,17.203125,18.296875
8,402.9,USR0000CPNN,20.000000,21.093750,21.093750,24.406250,25.000000,23.906250,22.796875,23.296875,...,12.203125,13.898438,15.601562,12.203125,15.000000,15.601562,16.093750,10.000000,9.398438,12.796875
9,111.3,USC00045123,13.898438,11.101562,8.898438,11.703125,10.000000,10.000000,11.101562,10.601562,...,14.398438,11.101562,10.000000,8.898438,7.800781,10.000000,7.199219,5.601562,8.898438,12.203125


In [3]:
df_test4=df_test3.iloc[:,2:]
result=df_test4.mean(axis=1)
type(result)

pandas.core.series.Series

In [4]:
df_test6=df_test3.iloc[:,:2]

In [5]:
df_test5=result.to_frame(name='Avg TMAX')
df_test5.shape

(2179, 1)

In [6]:
reg_data=pd.concat([df_test6, df_test5], axis=1)

In [7]:
reg_data.columns = ['Elevation', 'Station', 'Avg TMAX']
reg_data

,Elevation,Station,Avg TMAX
0,402.9,USR0000CPNN,25.281250
1,402.9,USR0000CPNN,25.453125
2,402.9,USR0000CPNN,25.406250
3,402.9,USR0000CPNN,25.546875
4,402.9,USR0000CPNN,24.921875
5,402.9,USR0000CPNN,25.921875
6,402.9,USR0000CPNN,26.109375
7,402.9,USR0000CPNN,26.171875
8,402.9,USR0000CPNN,25.734375
9,111.3,USC00045123,18.953125


In [8]:
import statsmodels.api as sm

## Regression

In [9]:
from pandas.stats.api import ols
regression = ols(y=reg_data['Avg TMAX'], x=reg_data['Elevation'])
regression

/Users/mengtingwang/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2881: FutureWarning: The pandas.stats.ols module is deprecated and will be removed in a future version. We refer to external packages like statsmodels, see some examples here: http://www.statsmodels.org/stable/regression.html
  exec(code_obj, self.user_global_ns, self.user_ns)



-------------------------Summary of Regression Analysis-------------------------

Formula: Y ~ <x> + <intercept>

Number of Observations:         2179
Number of Degrees of Freedom:   2

R-squared:         0.0324
Adj R-squared:     0.0320

Rmse:              2.4146

F-stat (1, 2177):    72.9849, p-value:     0.0000

Degrees of Freedom: model 1, resid 2177

-----------------------Summary of Estimated Coefficients------------------------
      Variable       Coef    Std Err     t-stat    p-value    CI 2.5%   CI 97.5%
--------------------------------------------------------------------------------
             x    -0.0020     0.0002      -8.54     0.0000    -0.0024    -0.0015
     intercept    21.5518     0.0593     363.45     0.0000    21.4356    21.6680
---------------------------------End of Summary---------------------------------

In [19]:
#sc.stop()